In [1]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=i7LjCUHwwd0j2mTPdf5HwH74FMhgNVre3VNwfAZh"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-04-19T12:18:02+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [2]:
import json

data = json.loads(response.text)
complexes = data['complexes']

# Create complex list
complex_list = []
for comp in complexes:
    complex_list.append({
        "complex_id": comp["id"],
        "complex_name": comp["name"]
    })

# Create venues list
venues_list = []
for comp in complexes:  # Loop through the complexes
    for venue in comp.get('venues', []):  # Safely access 'venues' within each complex
        venues_list.append({
            "venue_id": venue["id"],
            "venue_name": venue["name"],
            "city_name": venue.get("city_name", ""),
            "country_name": venue.get("country_name", ""),
            "country_code": venue.get("country_code", ""),
            "timezone": venue.get("timezone", ""),
            "complex_id": comp["id"]
        })


In [3]:
import pandas as pd
venues_df = pd.DataFrame(venues_list)
print(venues_df)

            venue_id                                  venue_name  \
0     sr:venue:70045                              Cancha Central   
1     sr:venue:74856                                Centre Court   
2     sr:venue:74858                                   Court One   
3      sr:venue:1496                                     COURT 1   
4      sr:venue:1500                                CENTER COURT   
...              ...                                         ...   
3322  sr:venue:82386          Loughborough University - Court 07   
3323  sr:venue:82388          Loughborough University - Court 06   
3324  sr:venue:82390          Loughborough University - Court 08   
3325  sr:venue:82392          Loughborough University - Court 05   
3326  sr:venue:82419  National Tennis Center - Outdoor - Court 1   

             city_name country_name country_code          timezone  \
0             Santiago        Chile          CHL  America/Santiago   
1              Seville        Spain        

In [4]:
import pandas as pd
complexes_df = pd.DataFrame(complex_list)
print(complexes_df)

           complex_id                           complex_name
0      sr:complex:705                               Nacional
1     sr:complex:1078                  Estadio de la Cartuja
2     sr:complex:1495                            Sibur Arena
3     sr:complex:2375             Complexo de Tenis do Jamor
4     sr:complex:4032  Shree Shiv Chhatrapati Sports Complex
..                ...                                    ...
650  sr:complex:82335                   Sarasota Sports Club
651  sr:complex:82343                 Le Central Tennis Club
652  sr:complex:82356              Dan Maskell Tennis Centre
653  sr:complex:82415     George P Mitchell 40 Tennis Center
654  sr:complex:82417                 National Tennis Center

[655 rows x 2 columns]


In [ ]:
complexes = data.get('complexes', [])
print(f"Number of complexes: {len(complexes)}")


total_venues = 0
for complex in complexes:
    total_venues += len(complex.get('venues', []))

print(f"Total number of venues: {total_venues}")

Number of complexes: 655
Total number of venues: 3327


In [6]:
!pip install mysql-connector-python

In [7]:
import mysql.connector

In [8]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # port=3306
)


In [9]:
mycursor = mydb.cursor(buffered=True)

In [10]:
mycursor.execute("SHOW DATABASES")
print(mycursor)

CMySQLCursorBuffered: SHOW DATABASES


In [11]:
mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)


('guvi',)
('information_schema',)
('mdtm40',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('practise',)
('project',)
('sportsradar',)
('test',)


In [18]:
mycursor.execute("CREATE DATABASE project")

In [12]:
mycursor.execute("USE project")

In [20]:
mycursor.execute("""
    CREATE TABLE complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    )
""")

In [21]:
mycursor.execute("""
    CREATE TABLE venues (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        timezone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY(complex_id) REFERENCES complexes(complex_id)
    )
""")

In [ ]:
insert_query = """
INSERT IGNORE INTO complexes (complex_id, complex_name)
VALUES (%s, %s)
"""


selected_columns = complexes_df[['complex_id', 'complex_name']]


array_values = selected_columns.values

data = []
for row in array_values:
    data.append(tuple(row))  


mycursor.executemany(insert_query, data)
mydb.commit()

print(mycursor.rowcount, "rows inserted into 'complexes' table.")

0 rows inserted into 'complexes' table.


In [ ]:
insert_query = """
INSERT IGNORE INTO venues (
    venue_id, venue_name, city_name, country_name,
    country_code, timezone, complex_id
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

selected_columns = venues_df[[
    'venue_id', 'venue_name', 'city_name', 'country_name',
    'country_code', 'timezone', 'complex_id'
]]

data = [tuple(row) for row in selected_columns.values]


mycursor.executemany(insert_query, data)
mydb.commit()
print(mycursor.rowcount, "rows inserted into 'venues' table.")


3325 rows inserted into 'venues' table.


In [19]:
from tabulate import tabulate

In [20]:
#List all venues along with their associated complex name

mycursor.execute("""
SELECT v.venue_name, c.complex_name
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
""" )
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------------------------------------------------+------------------------------------------------------------+
| venue_name                                                 | complex_name                                               |
+============================================================+============================================================+
| Center Court                                               | Kindarena                                                  |
+------------------------------------------------------------+------------------------------------------------------------+
| Amelie Mauresmo                                            | Kindarena                                                  |
+------------------------------------------------------------+------------------------------------------------------------+
| Celine Dumerc                                              | Kindarena                                                  |
+-------

In [21]:
#Count the number of venues in each complex

mycursor.execute("""
SELECT c.complex_name, COUNT(v.venue_id) AS venue_count
FROM complexes c
JOIN venues v ON c.complex_id = v.complex_id
GROUP BY c.complex_name
""" )
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------------------------------------------------+---------------+
| complex_name                                               |   venue_count |
+============================================================+===============+
| A.S.D. Tennis Club Como                                    |             9 |
+------------------------------------------------------------+---------------+
| AAJB Tennis                                                |            11 |
+------------------------------------------------------------+---------------+
| Abama Tennis Academy                                       |             7 |
+------------------------------------------------------------+---------------+
| Academy Zhangjiagang                                       |             7 |
+------------------------------------------------------------+---------------+
| Acapulco Princess Mundo Imperial                           |             3 |
+---------------------------------------------------

In [22]:
#Get details of venues in a specific country (e.g., Chile)
mycursor.execute("""
SELECT * FROM venues
WHERE country_name = 'Chile'
""" )
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------+-------------------------+--------------+----------------+----------------+------------------+------------------+
| venue_id       | venue_name              | city_name    | country_name   | country_code   | timezone         | complex_id       |
+================+=========================+==============+================+================+==================+==================+
| sr:venue:13830 | Cancha 1                | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
+----------------+-------------------------+--------------+----------------+----------------+------------------+------------------+
| sr:venue:13832 | Cancha 3                | Santiago     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
+----------------+-------------------------+--------------+----------------+----------------+------------------+------------------+
| sr:venue:15858 | Centre Court            | Santiago     | Chile          |

In [29]:
#Identify all venues and their timezones
mycursor.execute( """
SELECT venue_name, timezone FROM venues
""" )
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------------------------------------------------+--------------------------------+
| venue_name                                                 | timezone                       |
+============================================================+================================+
| Estadio Monumental                                         | America/Lima                   |
+------------------------------------------------------------+--------------------------------+
| Stadium                                                    | Asia/Shanghai                  |
+------------------------------------------------------------+--------------------------------+
| Court 2                                                    | Asia/Shanghai                  |
+------------------------------------------------------------+--------------------------------+
| Court 7                                                    | Australia/Sydney               |
+---------------------------------------

In [30]:
 #Find complexes that have more than one venue
mycursor.execute( """
SELECT c.complex_name, COUNT(v.venue_id) AS venue_count
FROM complexes c
JOIN venues v ON c.complex_id = v.complex_id
GROUP BY c.complex_id
HAVING COUNT(v.venue_id) > 1
""" )
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------------------------------------------------+---------------+
| complex_name                                               |   venue_count |
+============================================================+===============+
| Kindarena                                                  |             4 |
+------------------------------------------------------------+---------------+
| Estadio de la Cartuja                                      |             2 |
+------------------------------------------------------------+---------------+
| Estadio German Becker                                      |             4 |
+------------------------------------------------------------+---------------+
| Palais des sports de Gerland                               |             7 |
+------------------------------------------------------------+---------------+
| Sibur Arena                                                |             4 |
+---------------------------------------------------

In [31]:
#List venues grouped by country
mycursor.execute("""
SELECT country_name, GROUP_CONCAT(venue_name) AS venues
FROM venues
GROUP BY country_name
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
#Find all venues for a specific complex (e.g., Nacional)
mycursor.execute("""
SELECT v.*
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
WHERE c.complex_name = 'Nacional'
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------+----------------+-------------+----------------+----------------+------------------+----------------+
| venue_id       | venue_name     | city_name   | country_name   | country_code   | timezone         | complex_id     |
+================+================+=============+================+================+==================+================+
| sr:venue:70045 | Cancha Central | Santiago    | Chile          | CHL            | America/Santiago | sr:complex:705 |
+----------------+----------------+-------------+----------------+----------------+------------------+----------------+
